<a href="https://colab.research.google.com/github/rhumtea/trainModelML/blob/main/phong_alexnet_like_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Introduction:
- This project designs and train the Deep Learnign model.
- I choose the Natural Inmages with 8 classes from Kaggle.
- Link: https://www.kaggle.com/datasets/prasunroy/natural-images

In [25]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub

# Download the dataset from Kaggle and save in Google Colab Files
!kaggle datasets download -d prasunroy/natural-images

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
natural-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
# Unzip the natural-images.zip
with zipfile.ZipFile('natural-images.zip', 'r') as zip_ref:
  zip_ref.extractall('')

# Define path for original dataset
data_dir = 'natural_images'

In [27]:
import shutil
import random

# Define path for split data
base_dir = 'data_split'

# Define path for train, validation and test directory
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [28]:
# Split ratio
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [51]:
# Split data
categories = os.listdir(data_dir)

for category in categories:
  category_path = os.path.join(data_dir, category)

  if not os.path.isdir(category_path): continue

  # Create subdirectories for each class in train and val files
  os.makedirs(os.path.join(train_dir, category), exist_ok=True)
  os.makedirs(os.path.join(val_dir, category), exist_ok=True)
  os.makedirs(os.path.join(test_dir, category), exist_ok=True)

  # Get all images in the category to split
  images = os.listdir(category_path)
  random.shuffle(images)

  # Find indices of image to split
  total_images = len(images)
  train_images_index = int(train_ratio * total_images)
  val_images_index = int((train_ratio + val_ratio) * total_images)

  # Split data by indices
  train_images = images[:train_images_index]
  val_images = images[train_images_index:val_images_index]
  test_images = images[val_images_index:]

  # Move images to train and val directories with folders as original file:
  for image in train_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(train_dir, category))

  for image in val_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(val_dir, category))

  for image in test_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(test_dir, category))

In [30]:
# Define train generator which includes augmentation to expand the dataset
# and make the model more roburst
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

In [52]:
# Define validation and test generators with normalization
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [32]:
# Set image height, width for resizing image from th original size (uniformity)
# Set batch_size for deciding how many images per batch during train and validation
img_height, img_width = 227, 227
batch_size = 32

In [33]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical' # because of one-hot encode labels
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 6692 images belonging to 8 classes.
Found 2670 images belonging to 8 classes.
Found 2670 images belonging to 8 classes.


In [53]:
# Test_save generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 1039 images belonging to 8 classes.


2. Check before create Model

In [39]:
# Get the number of classes
num_classes = len(train_generator.class_indices)
print(num_classes)
class_names = [item for item in train_generator.class_indices]
print(class_names)

8
['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']


3. AlexNet50 - like - first try

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Build CNN model
model = Sequential([
            Conv2D(24, kernel_size=11, strides=4, padding='same', activation='relu', input_shape=(img_width, img_height, 3)),
            MaxPooling2D(pool_size=3, strides=2),
            BatchNormalization(),
            Conv2D(64, kernel_size=5, padding='same', activation='relu'),
            MaxPooling2D(pool_size=3, strides=2),
            BatchNormalization(),
            Conv2D(96, kernel_size=3, padding='same', activation='relu'),
            Conv2D(96, kernel_size=3, padding='same', activation='relu'),
            Conv2D(64, kernel_size=3, padding='same', activation='relu'),
            MaxPooling2D(pool_size=3, strides=2),
            Flatten(),
            Dense(128, activation= 'relu'),
            Dense(128, activation= 'relu'),

            Dense(num_classes, activation= 'softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4. Model for natural images

In [41]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 57, 57, 24)          │           8,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 28, 28, 24)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 24)          │              96 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 28, 28, 64)          │          38,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 13, 13, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 13, 13, 96)          │          55,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 13, 13, 96)          │          83,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 13, 13, 64)          │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 553,928 (2.11 MB)

 Trainable params: 553,752 (2.11 MB)

 Non-trainable params: 176 (704.00 B)

In [44]:
# Compile the model
model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [45]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.5895 - loss: 1.1342 - val_accuracy: 0.5996 - val_loss: 1.0146
Epoch 2/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.7367 - loss: 0.7137 - val_accuracy: 0.5633 - val_loss: 1.2974
Epoch 3/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.7761 - loss: 0.6326 - val_accuracy: 0.7217 - val_loss: 0.7872
Epoch 4/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.7931 - loss: 0.5771 - val_accuracy: 0.8476 - val_loss: 0.4154
Epoch 5/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 260s 1s/step - accuracy: 0.8223 - loss: 0.4917 - val_accuracy: 0.7876 - val_loss: 0.6546
Epoch 6/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.8392 - loss: 0.4310 - val_accuracy: 0.7865 - val_loss: 0.6306
Epoch 7/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.8211 - loss: 0.4784 - val_accuracy: 0.8588 - val_loss: 0.4110
Epoch 8/20
210/210 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.8442 - loss: 0.4110 - val_accu

-- Save the Model --


In [46]:
# Save and Reuse Model
model.save('alexnet_like_a.keras')

In [ ]:
# Load the model if needed
# from tensorflow.keras.models import load_model
# model = load_model('alexnet_like_a.keras')

5. Evaluate Model

In [54]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 240ms/step - accuracy: 0.9121 - loss: 0.2371
Test Accuracy: 92.11%


In [55]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

# Get the ground truth labels and predictions
y_true = test_generator.classes  # True labels from the test generator
y_pred = model.predict(test_generator)  # Predicted probabilities

# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute Precision, Recall, and F1-Score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys()))


33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 206ms/step
Precision: 13.06%
Recall: 13.09%
F1-Score: 13.06%

Classification Report:
              precision    recall  f1-score   support

    airplane       0.16      0.16      0.16       110
         car       0.17      0.16      0.16       146
         cat       0.13      0.14      0.14       133
         dog       0.13      0.11      0.12       106
      flower       0.09      0.08      0.08       127
       fruit       0.15      0.15      0.15       150
   motorbike       0.07      0.08      0.07       119
      person       0.14      0.15      0.15       148

    accuracy                           0.13      1039
   macro avg       0.13      0.13      0.13      1039
weighted avg       0.13      0.13      0.13      1039

